In [236]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import os 
import ast
import json
import time

https://stackoverflow.com/questions/62335537/scraping-text-from-kickstarter-projects-return-nothing

In [173]:
base = "/Users/ivankoh/Downloads/kickstarter data/"
filepaths = [base + f for f in os.listdir(base) if f.endswith('.csv')]
df = pd.concat(map(pd.read_csv, filepaths))
df_1 = df.drop_duplicates(subset=['id'])
df_1['main_url'] = df_1['urls'].apply(lambda x: eval(x)['web']['project'])
print(len(df))
print(len(df_1))

392244
203510


/var/folders/dq/33rpc0cx4f54t0vx2cdcgczr0000gn/T/ipykernel_74056/4110031066.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['main_url'] = df_1['urls'].apply(lambda x: eval(x)['web']['project'])


In [ ]:
def get_slug(url):
    return re.search('/projects/(.*)\?', url).group(1)

# get the slug from the urls. the slug is the input to the GraphQL API call 
slugs = list(map(get_slug, df_1['main_url'].tolist()))
# while iterating through all projects, add the story and risk to these lists. At the end len of both list 
# should be the same as the len of the dataframe. if all good then add as columns to the dataframe df. 
story_list = []
risk_list = []

In [269]:
block_size = len(df_1) // 100
num_blocks = len(df_1) // 2035
num_blocks
2 * block_size
print(0 * block_size, 1 * block_size)

0 2035


In [291]:
# instantiate a new Session 
s = requests.Session()
headers = {
            'content-type': 'application/json',
            'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0',
           'cookie': '_ksr_session=fc2U7qXXaRN91foNiE53NyU3s181NZO0Ll57xPkYxZ5iyUNgus35a0HwsPBTfViBY%2ByAKbtpRirAVLxOGKzG%2BYMOmsLRBPujZep%2Fca%2B1%2FXzW3xX56VXkh5w6ItYhIctEFifQQhw3rTmvoljyHw%3D%3D--4pK6xBEgChjqgmte--LH4Q1qSnhU%2FYX9JgTzGuSQ%3D%3D;'
          }
# GET request to kickstarter to get some token needed for later 
# r = s.get("https://www.kickstarter.com", headers=headers)
# BeautifulSoup parses the html returned as a response from the GET request 
# soup = BeautifulSoup(r.text, 'html.parser')
# xcsrf = soup.find("meta", {"name": "csrf-token"})["content"]

query = """
query Campaign($slug: String!) {
  project(slug: $slug) {
    risks
    story(assetWidth: 680)
  }
}"""

# PARAMS
counter = 0
block_number = 0

# pause for 5 seconds every 5 iterations
for i in range(105, (block_number + 1) * block_size):
    slug = slugs[i]
    if counter == 5:
        print("TAKING A NAP ZZZ...")
        time.sleep(60)
        counter = 0
    counter += 1
    print(f"--------{slug}------")
    # POST request to GraphQL API situated in kickstarter domain 
    # response from POST request should contain the story and risks
    r = s.post("https://www.kickstarter.com/graph",
        headers= {
              'content-type': 'application/json',
              'x-csrf-token': 'GgpaCc5tY-2VRVj1bE1aLOPKLkwRFEbIF205o2WorHLvaUEoOIPiZJSx4rbQg0csVbwBfq-jKGXPw2j42dvQCQ',
              'user-agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.111 Safari/537.36',
              'cookie': '_ksr_session=aVeEJvdlSYzJIE8X7f3v85C2T7b9Q30EN0ONWdeWjMLei6H06q6eRA3DZfCsbal40AfBTSbfczmlTv2ZIpvdyykn4QQYZdSlIzlalRivb6Xlb%2BXhXDhBNJg7KPYEEyFFIX3Qz8%2F61YNIoV136X5ju%2BE7jrqQFsGEB1%2FhAwJZ7kjD6QiwTDFdWWmJDr2wmREspLGXKUO0gicnsrUxQJPdTQMCcFNXy6sx95x5S%2FiNs4xVq0t4jWv%2BaiSnwBp8pC6Xu81FBhpD%2F9sNLZ9YyS5Obzbv0jI%3D--tM1UdSPo8XKxtj%2Bt--WQhnQX4EovqKwzBU9tiWIg%3D%3D'
        },
        json = {
            "operationName":"Campaign",
            "variables":{
                "slug": slug
            },
            "query": query
        })
    try:
        result = r.json()
        print("-------STORY--------")
        story_html = result["data"]["project"]["story"]
        soup = BeautifulSoup(story_html, 'html.parser')
        curr_story = []
        for i in soup.find_all('p'):
            print(i.get_text())
            curr_story.append(i.get_text())
        story_list.append(" ".join(curr_story) if curr_story else "")

        print("-------RISKS--------")
        print(result["data"]["project"]["risks"])
        risk_list.append(result["data"]["project"]["risks"] if result["data"]["project"]["risks"] else "")
    except:
        print(r)
        story_list.append("")
        risk_list.append("")
        continue

--------jennhales/raccoons-a-fox-and-an-octopus-in-the-bathtub------
-------STORY--------
I have three new paintings that I’m really excited about and I want to create a run of prints of each of them. 
The printer I’d like to use is Joseph C Woodard. They’re a family run 
business- which employs local artists and the sons and the daughter of 
the original founder. They have a large offset printer and they’ll be 
able to make beautiful (18" x 24") poster size prints.
The printer will use
 80 pound acid-free paper that contains 10 percent post-consumer 
content- the paper is FSC certified and archival quality and the inks 
are vegetable based. 
I'll ship your prints rolled in protective acid-free paper and placed in a sturdy cardboard tube.
Other rewards include:
These cards measure 4.25" x 5.25" each. I'll package all nine up together and ship you the bundle of them. 
These large postcards measure (5"x7"). I'll sign the back, add a stamp, and mail it to you. 
Both the greeting cards and

-------STORY--------
Cast Party Nightmare is a short  70s-80s style slasher film, that follows a cast of high school grads. After their final theater performance, they decide to get a few motel rooms to hold their senior cast party before each going their separate ways. Little do they know that they're being picked off one by one, while they're distracted by their interpersonal conflicts. 
 A screenwriter wrote our script and we're also currently working on some digitally drawn posters inspired by 1970s and 80s Horror Movie posters. Almost everyone working behind the scenes of this film is a college student, or recent high school graduate.  And we were fortunate enough to have a screenwriter write our script.  
Shooting for the film is set to take place in June 2021. 
This is an all-or-nothing fundraiser so if we don't reach our full goal, we will not be able to go through with our project — so we've got a lot of work to do.
The funds of this film will be going towards:
-------RISKS---

-------STORY--------
Para mi, como para muchos, el veganismo es amar a los millones de seres con los que
 convivimos en el planeta y no compartir esas horribles costumbres en las que se explotan animales a diario. Por ello, al entender que cada día somos más personas las que creemos en ese mundo más justo, he decidido crear una web para ayudar a distribuir esos productos que a día de hoy son difíciles de conseguir en tiendas o si lo hacen es a un precio muy elevado.
Por ello quiero crear este proyecto llamado "Mi caja vegana" en el que mediante suscripción o compras puntuales se puedan adquirir diversos tipos de cajas con variedad de productos alternativos a los tradicionales.
El dinero se usará para poder comenzar a tener un stock de productos, para el diseño y mantenimiento de la página web, así como para la elaboración de recetas que se incluirán dentro de las cajas.
Espero contar con tu apoyo para difundir esta forma de vida de la que cada vez más personas somos partícipes.
-------

-------STORY--------
Bitchtopia Magazine was started as a college project, looking to give feminist voices a platform to educate. We've shared important literature, interviewed inspiring women, published moving art, and offered up a feminist prospective that readers could relate to. You may be asking, why change it? 
Though Bitchtopia has seen immense success since its inception, it's time for us to graduate. Each of our contributors, editors, and social media mavens have worked extremely hard and have seen so much personal growth. It's only fair that our publication grow with them.
Rose Water Magazine will still be feminist. We hope that our art and commentary through the feminist lens will trickle down and nurture and educate our readers. Rose Water will be intersectional, thought-provoking, and creative. 
With more funding, we will be able to hold workshops for our current and new contributors and also offer them compensation for their published work.   Our funding will also be used

-------STORY--------
Hey y'all!
I'm starting a new underground, DIY publishing company based out of my van, which is also my home.  I've run underground presses before, and i LOVE helping artists, writers, graphic novelists, and other create types spread their work.  I'm interested in stories that go against the grain, stories that illuminate lives most people do not have access to.
I'm going to create thematic zines, for example, one theme will be Weird Sex, and another will be What to do With your Labia: an Off Grid Guide to your P*ssy
I'm also going to hold open submissions for artists who want their work published.
Then I am going to construct the zines, print them, and distribute them.
I think the best way to end ignorance and fear based living is through getting to know folks who are different from us, and the most enjoyable way to access other people's lives is through art.  
I want to publish poetry, prose, fiction, creative non fiction, comics, graphic art, rants, philosophy, 

...

-------STORY--------
Here we are, two years on from the creation of the first Providero Tea and Coffee House. It’s been a wonderful ride to this point! We come to you now to ask you to join with us again in a new adventure!
But first, here’s a recap…..
With the help and support of our amazing Kickstarter Community, our dreaming and hard work culminated in the opening of the Providero Tea and Coffee House in Llandudno Junction in January 2014. 
In two years we have served thousands of cups of the finest tea and coffee on the North Wales coast, alongside our famous home-made cakes and locally-baked bread. 
We have continued to develop a wonderful customer base of regulars, many of whom have become friends with us, and with each other. 
We have served visitors from all over the world and welcomed some fascinating characters through our door - it’s been a privilege to offer this hospitality in our little corner of beautiful North Wales. 

Our team of 2 (Jon and Ellie!) has grown, to includ

-------STORY--------
FIC VALORES 
Es un proyecto que inspira a muchas vidas
 y les ayuda a ser mejor persona,
 utilizando como principal medio, las obras cinematográficas que se exhiben en el festival. 
Marcando positivamente, no sólo a los asistentes, sino también a sus creadores y protagonistas
 a través de sus personajes e historias. 
Todas las películas que se proyectan durante el festival, contienen valores humanos universales. Consolidandóse así, como un proyecto único
 en toda Latinoamérica. 
-------RISKS--------
A lo largo de los años no hemos podido ser beneficiados por algún programa cultural del Gobierno del Estado ni del Gobierno Federal, para nuestra asociación es importante seguir difundiendo todos los valores humanos a través del Cine. Queremos seguir siendo una venta de proyección  cinematográfico para jóvenes creadores.
--------clacpi/ficmayab------
-------STORY--------
FICMAYAB’ - Cine Indígena para el Mundo 
En el mes de octubre de 2018, CLACPI, la Coordinado

-------STORY--------
[English below]
1. Cos’è il Fantafestival? Un po’ di storia…
Il Fantafestival è il più longevo festival italiano dedicato al cinema fantastico in tutte le sue declinazioni: fantascienza, horror, fantasy, paranormale, gotico e altri generi affini, con alle spalle ben 37 edizioni! Probabilmente molti già lo conosceranno, ma è giusto raccontare un po’ della sua storia, per tutti coloro che non hanno ancora avuto modo di sperimentarlo in prima persona. Il Fantafestival nasce a Roma nel 1981, come diretta derivazione delle rassegne di cinema di fantascienza organizzate per il circuito Italnoleggio negli anni ’74-’77 e delle grandi kermesse di cinema fantastico con le quali il Cineclub Tevere di Roma connotò la propria programmazione a partire dal ’75. Iniziato al cinema Clodio nell’ottobre del 1981 e poi passato, negli anni, come sede principale prima al complesso Capranica-Capranichetta, poi al Barberini, al Quattro Fontane e al Savoy, il festival si è sempre caratteri

-------STORY--------
Expedition to the Arctic Circle 
I am embarking on a photographic expedition to the western shores of Greenland along the southern edge of the Arctic Circle.  My name is Emanuel Dimitri Volakis and I have been creating black and white photographs for over 27 years. My interest in photography began when I was eleven years old. My professional career as a fine artist commenced in my twenties. I have been working full time as an independent artist for almost three decades.  
My past work has focused on the aesthetics and achievements of humanity from ancient times to the modern age.  I have photographed the ruins at Pompeii, created an extensive body of work on the Gardens at Versailles and completed additional series on baroque sculpture, architecture and the urban landscape throughout Western Europe and North America.
My photographic travels have taken me to many unique regions across North America including Yosemite, Mineral King,  the Olympic Peninsula, the northe

KeyboardInterrupt: 

## Latest Iteration Ran Till: 274, Local copy latest: 0_274

In [294]:
#SAVE LIST IN LOCAL FOR USE LATER. NAME FILE AS {from}_{to]_{story/risk}.json
story_file_name = "0_274_story" #CHANGE THIS
risk_file_name = "0_274_risk" #CHANGE THIS 

story_file_name += ".json"
risk_file_name += ".json"

with open(story_file_name, 'w') as f:
    # indent=2 is not needed but makes the file human-readable 
    # if the data is nested
    json.dump(story_list, f, indent=2) 

with open(risk_file_name, 'w') as f:
    # indent=2 is not needed but makes the file human-readable 
    # if the data is nested
    json.dump(risk_list, f, indent=2) 
    
# with open(story_file_name, 'r') as f:
#     story_subset = json.load(f)

In [292]:
print("risk:", len(risk_list),"\n", "story:", len(story_list))

risk: 275 
 story: 275


In [293]:
print("LATEST CALL DF ROW", "\n")
print(df_1.iloc[274]['blurb'])

print("LATEST STORY IN LIST", "\n")
print(story_list[-1])

print("LATEST RISK IN LIST", "\n")
print(risk_list[-1])

LATEST CALL DF ROW 

A Hard Enamel Keychain of Mr. Drippy from the PS3 game Ni No Kuni
LATEST STORY IN LIST 

Hello, My name is Daryl and I run a small Etsy store where I sell enamel pins and other products of my designs. Typically, I try to focus on designing and selling original characters, but I recently played through the PS3 classic Ni No Kuni and was inspired to design my first keychain.  Here are some specifics: Assurance: All pledge money received will go straight into the production and manufacturing of this keychain. The manufacturer that I work with does require 100 piece minimum, so I don't foresee having much money leftover. If there does happen to be some funds left over, all the remainder will go toward purchasing new backing cards and other such items for packaging.
LATEST RISK IN LIST 

I've been designing and manufacturing hard enamel pins for almost a year now and have yet to encounter any sort of challenge that wasn't easily overcome. However, this being my first ke

In [165]:
s = requests.Session()

In [234]:
import requests
import json

url = "https://www.kickstarter.com/graph"

payload = json.dumps([
  {
    "operationName": "Campaign",
    "variables": {
      "slug": "leightonconnor/akashic-titan-blue-bolt"
    },
    "query": "query Campaign($slug: String!) {\n  project(slug: $slug) {\n    id\n    isSharingProjectBudget\n    risks\n    story(assetWidth: 680)\n    currency\n    spreadsheet {\n      displayMode\n      public\n      url\n      data {\n        name\n        value\n        phase\n        rowNum\n        __typename\n      }\n      dataLastUpdatedAt\n      __typename\n    }\n    environmentalCommitments {\n      id\n      commitmentCategory\n      description\n      __typename\n    }\n    __typename\n  }\n}\n"
  }
])
headers = {
  'content-type': 'application/json',
  'x-csrf-token': 'bL5nMRm5s147N8RxvNcth-E8NAOt3SprGYtvvKsVLnuZ3XwQ71cy1zrDfjIAGTCHV0obMRNqRMbBJT7nF2ZSAA',
  'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0',
  'cookie': '_ksr_session=eus10UNK7iB%2FwIMXafXg5yJdYvRD7rHlR%2Ba1KEbUnB5EKTgQxNKEIqrp%2BIyjRC%2FNXw76u4Eki4AsiBom5M1LKNJKWCD65CBzeegWrk2%2FRV9Sht6dlOKplCATNdrmEIIa0Bt%2FKKGnT2m9ID1nTCduj73%2BdeJK663MNcbYWXBHkFyZJgUKh4JcCx3hL0UF2quVeBvfV45M0CNrT1j4DchvTlFBn%2BpE37ZaGldNz7BF0RAcCjIZrt%2FzNkxugCaFfWEPjxh9amOCvytIcdjxOaE68AAmHuU%3D--RPNmoWC%2FGnWTmsfw--9466UG%2Bqs2V%2F8B8zFtXUig%3D%3D'
}

response = requests.request("POST", url, headers=headers, data=payload)
print(response.status_code)
print(response.json()[0]['data']['project']['risks'])

403


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [256]:
import requests
import json

url = "https://www.kickstarter.com/graph"

payload = json.dumps([
  {
    "operationName": "Campaign",
    "variables": {
      "slug": "leightonconnor/akashic-titan-blue-bolt"
    },
    "query": "query Campaign($slug: String!) {\n  project(slug: $slug) {\n    id\n    isSharingProjectBudget\n    risks\n    story(assetWidth: 680)\n    currency\n    spreadsheet {\n      displayMode\n      public\n      url\n      data {\n        name\n        value\n        phase\n        rowNum\n        __typename\n      }\n      dataLastUpdatedAt\n      __typename\n    }\n    environmentalCommitments {\n      id\n      commitmentCategory\n      description\n      __typename\n    }\n    __typename\n  }\n}\n"
  }
])
headers = {
  'content-type': 'application/json',
  'x-csrf-token': 'GgpaCc5tY-2VRVj1bE1aLOPKLkwRFEbIF205o2WorHLvaUEoOIPiZJSx4rbQg0csVbwBfq-jKGXPw2j42dvQCQ',
  'user-agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.111 Safari/537.36',
  'cookie': '_ksr_session=aVeEJvdlSYzJIE8X7f3v85C2T7b9Q30EN0ONWdeWjMLei6H06q6eRA3DZfCsbal40AfBTSbfczmlTv2ZIpvdyykn4QQYZdSlIzlalRivb6Xlb%2BXhXDhBNJg7KPYEEyFFIX3Qz8%2F61YNIoV136X5ju%2BE7jrqQFsGEB1%2FhAwJZ7kjD6QiwTDFdWWmJDr2wmREspLGXKUO0gicnsrUxQJPdTQMCcFNXy6sx95x5S%2FiNs4xVq0t4jWv%2BaiSnwBp8pC6Xu81FBhpD%2F9sNLZ9YyS5Obzbv0jI%3D--tM1UdSPo8XKxtj%2Bt--WQhnQX4EovqKwzBU9tiWIg%3D%3D'
}

response = requests.request("POST", url, headers=headers, data=payload)
print(response.status_code)
print(response.json()[0]['data']['project']['risks'])

200
This is my fourth solo Kickstarter campaign, and I delivered all three of my previous games in a timely manner. 

I have a complete draft of the text, with the exception of Appendix 1. I have a team of editors lined up who will give me feedback so that I can revise, expand, and polish it as needed. The most time-consuming tasks will be completing the artwork and the layout. I've done this before, with the previous Akashic Titan zine and with my game Project 8Ball. Depending on the stretch goals I will be depending on someone else for illustrations and additional text, but I have worked with Josh before and I am confident he will hit his deadlines. 

I believe I will be able to finish the book by June, but just to be cautious I have set the estimated delivery month as July. 

Barring any unforeseen catastrophes, I should have no problem printing and shipping the zine by July.


In [255]:
s = requests.Session()
headers = {
            'content-type': 'application/json',
            'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0',
          }
# GET request to kickstarter to get some token needed for later 
r = s.get("https://www.kickstarter.com", headers=headers)
#BeautifulSoup parses the html returned as a response from the GET request 
soup = BeautifulSoup(r.text, 'html.parser')
xcsrf = soup.find("meta", {"name": "csrf-token"})["content"]

In [ ]:
'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0'
'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36'

In [237]:
print("hello")
time.sleep(3)
print("goodnight")

hello
goodnight
